In [64]:
from pathlib import Path
import numpy as np
from numba import jit
from utils.get_input import get_input
from io import StringIO
import pandas as pd


inp = Path("./test_input").read_text()
# inp = get_input(8)

In [65]:
# %%timeit
# Structure input
inp_np = np.array([list(line) for line in inp.split("\n")])

antennas = np.where(inp_np != '.')
antenna_types = inp_np[antennas]

data = np.array([antennas[0], antennas[1], antenna_types]).T
data = pd.DataFrame(data, columns=["y", "x", "type"])
data["x"] = data["x"].astype(int)
data["y"] = data["y"].astype(int)
data

,y,x,type
0,1,8,0
1,2,5,0
2,3,7,0
3,4,4,0
4,5,6,A
5,8,8,A
6,9,9,A


In [67]:
import itertools

max_y, max_x = inp_np.shape

antinodes = []
for n, t in data.groupby("type"):
    print(n)
    ant_in = t.index
    ant_combs = list(itertools.combinations(ant_in, 2))
    for ant1, ant2 in ant_combs:
        t_1 = t.loc[ant1]
        t_2 = t.loc[ant2]
        diff_x = t_1["x"] - t_2["x"]
        diff_y = t_1["y"] - t_2["y"]

        a_1 = (t_1["y"] + diff_y, t_1["x"] + diff_x, n)
        a_2 = (t_2["y"] - diff_y, t_2["x"] - diff_x, n)
        if a_1[0] >= 0 and a_1[0] < max_y and a_1[1] > 0 and a_1[1] < max_x:
            antinodes.append(a_1)
        if a_2[0] >= 0 and a_2[0] < max_y and a_2[1] > 0 and a_2[1] < max_x:
            antinodes.append(a_2)


inp_np2 = inp_np.copy()


for x,y, n in antinodes:
    inp_np2[x,y] = "#"

print(len(np.unique(antinodes)))
inp_np2


    
    

0
A
12


array([['.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '#', '.', '.', '.', '.', '0', '.', '.', '.'],
       ['.', '.', '.', '.', '#', '0', '.', '.', '.', '.', '#', '.'],
       ['.', '.', '#', '.', '.', '.', '.', '0', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '0', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.']],
      dtype='<U1')

In [60]:
@jit(nopython=True)
def part1():
    col_1 = np.sort(inp_np[0])
    col_2 = np.sort(inp_np[1])

    diff = np.abs(col_1 - col_2)
    return diff.sum()


part1()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function sub>) found for signature:
 
 >>> sub(array([unichr x 1], 1d, C), array([unichr x 1], 1d, C))
 
There are 14 candidate implementations:
[1m  - Of which 10 did not match due to:
  Overload of function 'sub': File: <numerous>: Line N/A.
    With argument(s): '(array([unichr x 1], 1d, C), array([unichr x 1], 1d, C))':[0m
[1m   No match.[0m
[1m  - Of which 2 did not match due to:
  Operator Overload in function 'sub': File: unknown: Line unknown.
    With argument(s): '(array([unichr x 1], 1d, C), array([unichr x 1], 1d, C))':[0m
[1m   No match for registered cases:
    * (int64, int64) -> int64
    * (int64, uint64) -> int64
    * (uint64, int64) -> int64
    * (uint64, uint64) -> uint64
    * (float32, float32) -> float32
    * (float64, float64) -> float64
    * (complex64, complex64) -> complex64
    * (complex128, complex128) -> complex128[0

In [ ]:
%%timeit
part1()

13.9 μs ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
@jit(nopython=True)
def part2():
    col_1 = inp_np[0]
    col_2 = inp_np[1]

    count_col_2 = np.bincount(col_2)
    counts = count_col_2[col_1]

    res = np.sum(col_1 * counts)
    return res


part2()

23963899

In [ ]:
%%timeit
part2()

8.22 μs ± 255 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
